In [8]:
import numpy as np
import pandas as pd
import re
import network as nx
import matplotlib.pyplot as plt
import itertools
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import RegexpStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

stop words

In [ ]:
english_stop_list = list(stopwords.words('english'))
print(english_stop_list)

combined 데이터 불러오기

In [4]:
combined = pd.read_csv('combined.csv')

In [6]:
combined.head()

,Unnamed: 0,title,score,id,url,num_comments,created_utc,selftext,flair,comments,sentiment
0,0,Korea introduces new point system for job seek...,36,9jw36t,https://www.youtube.com/watch?v=IrZHXBn8zLE,28,1538222833,NaN,Immigration,This is only makes getting a job seeking vis...,1.0
1,1,How to work legally in Korea on American citiz...,3,9jsxq6,https://www.reddit.com/r/korea/comments/9jsxq6...,22,1538186675,\nI’m planning on going there for school so I ...,Immigration,yeah hope ur conversational korean is good ...,1.0
2,2,How one man transformed South Korea s sordid ...,28,dry15c,https://www.reddit.com/r/korea/comments/dry15c...,6,1572954324,[https://www.cnbc.com/2019/11/05/how-yanolja-m...,Economy,You won t BELIEVE how this couple in lt your...,1.0
3,3,Does Korea have any words that describe landsc...,1,drww85,https://www.reddit.com/r/korea/comments/drww85...,9,1572946432,Hi I writing a novel about the natural history...,Nature,means stunning Oh ok thanks ...,1.0
4,4,Is it normal to give your parents allowances,9,drsn0w,https://www.reddit.com/r/korea/comments/drsn0w...,29,1572920637,I've overheard some foreign exchange students ...,Culture,I do that As I know most East asian and Sout...,1.0


명사인 단어 추출

In [9]:
# 키워드 빈도 추출 - 어떤 단어가 주로 나오는지
list4 = list(itertools.chain(*combined['title_tokenized']))
tokens_pos = nltk.pos_tag(list4)
print(tokens_pos)

KeyError: 'title_tokenized'

In [ ]:
# 명사는 NN을 포함하고 있음을 알 수 있음
NN_words = []
for word, pos in tokens_pos:
    if 'NN' in pos:
        NN_words.append(word)
print(NN_words)

In [ ]:
def flatten(l):
    flatList=[]
    for elem in l:
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList

In [ ]:
NN_words=flatten(NN_words)
NN_words=[x for x in NN_words if len(x)>2]
NN_words=[x for x in NN_words if not x.isdigit()]
# 상위 50개 단어 frequency 뽑기 
pd.Series(NN_words).value_counts().head(50)

In [ ]:
#직접 만든 불용어 사전의 용어 리스트에서 삭제

In [ ]:
customized_stopwords = ['http', 'amp', 'www', "ec", "ve",'eb','th','st'] # 직접 만든 불용어 사전

unique_NN_words1 = set(NN_words)
for word in unique_NN_words1:
    if word in customized_stopwords:
        while word in NN_words: NN_words.remove(word)

print(NN_words)

여행과 관련된 단어 추출

In [ ]:
travel_word = list(itertools.chain(*datas))

In [ ]:
NN_words=flatten(travel_word)
NN_words=[x for x in NN_words if len(x)>2]
NN_words=[x for x in NN_words if not x.isdigit()]
# 상위 50개 단어 frequency 뽑기 
pd.Series(NN_words).value_counts().head(50)

In [ ]:
#전체 명사 워드클라우드 그리기

import wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from os import path

noun_text = ''
for word in NN_words:
    noun_text = noun_text +' '+word

wordcloud = WordCloud(max_font_size=60, relative_scaling=.5).generate(noun_text) # generate() 는 하나의 string value를 입력 받음
plt.figure()
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

연관어 분석 전처리

In [ ]:
title_tokenized_upper3 = []
for post_word in combined['title_tokenized']:
    for word in post_word:
        if len(word) < 3:
            post_word.remove(word)
    title_tokenized_upper3.append(post_word)

In [ ]:
combined['title_tokenized_upper3'] = title_tokenized_upper3

In [ ]:
#품사 태깅 다시 해주기

title_tokenized_list = []
for post_word in combined['title_tokenized_upper3']:
    tag_list = nltk.tag.pos_tag(post_word)
    title_tokenized_list.append(tag_list)  

In [ ]:
combined['title_tokenized_NN'] = title_tokenized_list
combined['title_tokenized_NN']= combined['title_tokenized_NN'].apply(lambda x: [token for token, tag in x if tag in ('NN')])

In [ ]:
# combined 값 리스트 형식 변환

from ast import literal_eval
combined['title_tokenized'] = combined['title_tokenized'].fillna('[]')
combined['comments'] = combined['comments'].apply(literal_eval)
combined['clean_title'] = combined['clean_title'].apply(literal_eval)
combined['title_tokenized'] = combined['title_tokenized'].apply(literal_eval)
combined['title_tokenized_NN'] = combined['title_tokenized_NN'].apply(literal_eval)
combined.head()

In [ ]:
# num_comments가 3 이하인 post 삭제
for i in range(len(combined['title_tokenized'])):
    for word in combined['title_tokenized'][i]:
        if len(word) < 4:
            print(word)
            combined.iloc[i, -1].remove(word)

In [ ]:
#불용어 사전 추가 및 제거

#직접 만든 불용어 사전의 용어 리스트에서 삭제

customized_stopwords = ['http', 'amp', 'www', "ec", "ve",'eb','th','st','https','com','page','wiki','reddit'] # 직접 만든 불용어 사전

list_stop=[]
for post_word in combined['title_tokenized_NN']:
    for word in post_word:
        if word in customized_stopwords:
            while word in post_word: post_word.remove(word)
    list_stop.append(post_word)
    
combined['title_tokenized_NN_stop'] = list_stop

In [ ]:
combined.to_csv('./preprocessing.csv')

연관어 분석 - word2vec 사용

In [ ]:
from gensim.models.word2vec import Word2Vec

def model(keyword) :
    model=Word2Vec(combined['title_tokenized_NN_stop'],sg=1, window=10, min_count=1)
    model.init_sims(replace=True)
    result = model.wv.most_similar(keyword, topn = 50)
    return result

In [ ]:
words = ['travel', 'seoul', 'tour', 'korea', 'trip']

In [ ]:
#  'travel', 'seoul', 'tour', 'korea', 'trip' 적용

for i in words : 
    try :
        print(i)
        print(model(i))
        print()
    except : 
        print('해당 키워드는 출력할 수 없습니다.')
        print()

In [ ]:
# 변수설정

travel = model("travel")
seoul = model("seoul")
tour = model("tour")
korea = model("korea")
trip = model("trip")

연관어 워드클라우드

In [ ]:
def ass_wordcloud(ass) : 
    words = []
    for i, j in ass :
        words.append(i)
    
    cloud_data =" ".join(words)

    from wordcloud import WordCloud
    import matplotlib.pyplot as plt

    wordcloud = WordCloud(background_color='white').generate(cloud_data)

    plt.figure(figsize=(22,22)) #이미지 사이즈 지정
    plt.imshow(wordcloud, interpolation='lanczos') #이미지의 부드럽기 정도
    plt.axis('off') #x y 축 숫자 제거
    plt.show() 

In [ ]:
ass_wordcloud('travel')   

In [ ]:
ass_wordcloud('seoul')   

In [ ]:
ass_wordcloud('tour')   

In [ ]:
ass_wordcloud('korea')   

In [ ]:
ass_wordcloud('trip')   

특정 단어들 포함된 포스트 말뭉치 추출

In [ ]:
def post_words(keyword) : 
    
    words = []
    for i, j in ass :
        words.append(i)
        
    word_post_indexes = []
    for i in range(len(combined)) : 
        if (keyword in combined['title_tokenized_NN_stop'][i]) ==True : 
            for j in range(len(combined.iloc[i, -1])) : 
                if combined['title_tokenized_NN_stop'][i][j] in words : 
                    word_post_indexes.append(i)
                    break
                    
    word_post_datas = []
    for i in combined.iloc[word_post_indexes, -1] : 
        word_post_datas.append(i)
    
    return word_post_datas

네트워크 그리기 - 특정 단어 관련

In [11]:
import numpy as np
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def network(keyword) : 
    
    post_datas = post_words(keyword)
    
    from apyori import apriori

    result=(list(apriori(post_datas, min_support=0.2)))
    df=pd.DataFrame(result)

    df['length']=df['items'].apply(lambda x: len(x))
    df=df[(df['length']==2)&(df['support']>=0.05)].sort_values(by='support', ascending=False)
    
    G=nx.Graph()
    G.add_edges_from(df['items'])
    pr=nx.pagerank(G)
    nsize=np.array([v for v in pr.values()])
    nsize=2000*(nsize-min(nsize))/(max(nsize)-min(nsize))
    pos=nx.spring_layout(G)
    plt.figure(figsize=(16,12))
    plt.axis('off')
    nx.draw_networkx(G, font_size=10, pos=pos, node_color=list(pr.values()), node_size=nsize,
                     alpha=0.5, edge_color='.7', cmap=plt.cm.YlGn)

In [ ]:
netword('travel')

In [ ]:
netword('seoul')

In [ ]:
netword('tour')

In [ ]:
netword('korea')

In [ ]:
netword('trip')